https://discourse.julialang.org/t/how-to-reduce-large-memory-allocation-by-lu-with-sparse-matrices/64791/7

In [1]:
VERSION

v"1.6.2"

In [2]:
using BenchmarkTools
using Random; Random.seed!(4649373)
using LinearAlgebra
using SparseArrays
using SparseArrays: getcolptr
using SuiteSparse: decrement
using SuiteSparse.UMFPACK: UmfpackLU, UMFVTypes, UMFITypes, umfpack_numeric!

"""https://github.com/JuliaLang/SuiteSparse.jl/blob/master/src/umfpack.jl#L220-L278"""
function lu_org!(F::UmfpackLU, S::SparseMatrixCSC{<:UMFVTypes,<:UMFITypes}; check::Bool=true)
    zerobased = getcolptr(S)[1] == 0
    F.m = size(S, 1)
    F.n = size(S, 2)
    F.colptr = zerobased ? copy(getcolptr(S)) : decrement(getcolptr(S))
    F.rowval = zerobased ? copy(rowvals(S)) : decrement(rowvals(S))
    F.nzval = copy(nonzeros(S))

    umfpack_numeric!(F, reuse_numeric = false)
    check && (issuccess(F) || throw(LinearAlgebra.SingularException(0)))
    return F
end

"""A modification of https://github.com/JuliaLang/SuiteSparse.jl/blob/master/src/umfpack.jl#L220-L278"""
function lu_dot!(F::UmfpackLU, S::SparseMatrixCSC{<:UMFVTypes,<:UMFITypes}; check::Bool=true)
    zerobased = getcolptr(S)[1] == 0
    F.m = size(S, 1)
    F.n = size(S, 2)
    if zerobased
        F.colptr .= getcolptr(S)
        F.rowval .= rowvals(S)
    else
        F.colptr .= getcolptr(S) .- oneunit(eltype(S))
        F.rowval .= rowvals(S) .- oneunit(eltype(S))
    end
    F.nzval .= nonzeros(S)

    umfpack_numeric!(F, reuse_numeric = false)
    check && (issuccess(F) || throw(LinearAlgebra.SingularException(0)))
    return F
end

n = 2^10
X = I + sprandn(n, n, 1e-3)

F = lu(X)
G = lu(X)
@show lu(X).L == lu!(F, X).L
@show lu(X).U == lu!(F, X).U
@show lu!(F, X).L == lu_dot!(G, X).L
@show lu!(F, X).U == lu_dot!(G, X).U
println()
print("lu(X):        ")
@btime lu($X)
print("lu!(F, X):    ")
@btime lu!($F, $X)
print("lu_dot!(F, X):")
@btime lu_dot!($F, $X);

(lu(X)).L == (lu!(F, X)).L = true
(lu(X)).U == (lu!(F, X)).U = true
(lu!(F, X)).L == (lu_dot!(G, X)).L = true
(lu!(F, X)).U == (lu_dot!(G, X)).U = true

lu(X):          141.900 μs (61 allocations: 505.51 KiB)
lu!(F, X):      71.500 μs (36 allocations: 220.17 KiB)
lu_dot!(F, X):  73.800 μs (31 allocations: 179.58 KiB)


In [3]:
print("umfpack_numeric!(F, reuse_numeric = false):")
let F = F, S = X
    zerobased = getcolptr(S)[1] == 0
    F.m = size(S, 1)
    F.n = size(S, 2)
    if zerobased
        F.colptr .= getcolptr(S)
        F.rowval .= rowvals(S)
    else
        F.colptr .= getcolptr(S) .- oneunit(eltype(S))
        F.rowval .= rowvals(S) .- oneunit(eltype(S))
    end
    F.nzval .= nonzeros(S)

    @btime umfpack_numeric!(F, reuse_numeric = false)
end;

umfpack_numeric!(F, reuse_numeric = false):  66.000 μs (31 allocations: 179.58 KiB)


In [4]:
@which umfpack_numeric!(F, reuse_numeric = false)

(::SuiteSparse.UMFPACK.var"#umfpack_numeric!##kw")(::Any, ::typeof(umfpack_numeric!), U::UmfpackLU{Float64, Int64}) in SuiteSparse.UMFPACK at D:\Julia-1.6.2\share\julia\stdlib\v1.6\SuiteSparse\src\umfpack.jl:323

https://github.com/JuliaLang/SuiteSparse.jl/blob/master/src/umfpack.jl#L384-L400

In [5]:
mutable struct Factor F::UmfpackLU end
factor = Factor(F)
factor.F === F

true